# Fake Page View generator

This notebook generates page views based on a three-hour dataset.

In [1]:
%%bash
python3 -m pip install kafka-python

In [1]:
# Initialize Kafka Topics
from kafka import KafkaClient
from kafka.admin import KafkaAdminClient, NewTopic

TOPICS = ["clicks", "clicks-cleaned", "clicks-calculated", "clicks-calculated-ddos", "clicks-calculated-forbidden"]

kafka_client = KafkaClient("localhost:9092")
existing_topics = kafka_client.topic_partitions

admin_client = KafkaAdminClient(bootstrap_servers="localhost:9092")
topic_list = []
for topicname in TOPICS:
    if topicname not in existing_topics:
        print("Creating topic: {}".format(topicname))
        topic_list.append(NewTopic(name=topicname, num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

Creating topic: clicks
Creating topic: clicks-cleaned
Creating topic: clicks-calculated
Creating topic: clicks-calculated-ddos
Creating topic: clicks-calculated-forbidden


CreateTopicsResponse_v0(topic_errors=[(topic='clicks', error_code=0), (topic='clicks-calculated', error_code=0), (topic='clicks-cleaned', error_code=0), (topic='clicks-calculated-ddos', error_code=0), (topic='clicks-calculated-forbidden', error_code=0)])

In [3]:
# Extract archive of click data
import zipfile

ARCHIVEF = "20180113-6-9-with-nulls.json.zip"
CLICKSF = "20180113-6-9-with-nulls.json"

with zipfile.ZipFile(ARCHIVEF,"r") as zip_ref:
    zip_ref.extractall(".")

In [2]:
# SENDMODE = "kafka"
SENDMODE = "http"

In [4]:
# Send click data
import json
import time
from datetime import datetime

import requests
from kafka import KafkaProducer
from kafka.errors import KafkaError
from IPython.display import clear_output

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

orig_start = 0
replay_start = time.time() * 1000

counter = 0
i = 0

print("Simulating website traffic...")

with open(CLICKSF, 'r') as f:
    first = True
    start_ts = 0
    for line in f:
        click = json.loads(line)

        orig_event = click["ts_ingest"] or 0
        
        if orig_event:
            curr_ts = time.time() * 1000

            if first:
                first = False
                orig_start = click["ts_ingest"]

            difference = (orig_event - orig_start) - (curr_ts - replay_start)

            if difference > 0:
                print("sleeping {0:.2f} seconds".format(difference/1000.0))
                time.sleep(difference/1000.0)
            
        click['ts_ingest'] = round((orig_event - orig_start) + replay_start)

        if SENDMODE == "kafka":            
            future = producer.send('clicks', click)
            try:
                record_metadata = future.get(timeout=10)
            except KafkaError:
                log.exception()
                pass
        else:
            #print(f"sending {line}")
            requests.post("http://localhost:5000/clicks", data=json.dumps(click).encode("utf-8"), headers={'Content-Type':'application/json'})
        
        print("✓ {}".format(datetime.fromtimestamp(orig_event/1000.0)))
        if i > 1000:
            clear_output()
            i = 0
        i = i+1


✓ 2018-01-13 05:10:32.446000
✓ 2018-01-13 05:10:32.459000
✓ 2018-01-13 05:10:32.467000
✓ 2018-01-13 05:10:32.468000
✓ 2018-01-13 05:10:32.471000
✓ 2018-01-13 05:10:32.475000
✓ 2018-01-13 05:10:32.476000
✓ 2018-01-13 05:10:32.484000
✓ 2018-01-13 05:10:32.484000
✓ 2018-01-13 05:10:32.509000
✓ 2018-01-13 05:10:32.596000
✓ 2018-01-13 05:10:32.622000
✓ 2018-01-13 05:10:32.637000
✓ 2018-01-13 05:10:32.647000
✓ 2018-01-13 05:10:32.666000
✓ 2018-01-13 05:10:32.669000
✓ 2018-01-13 05:10:32.692000
✓ 2018-01-13 05:10:32.697000
✓ 2018-01-13 05:10:32.739000
✓ 2018-01-13 05:10:32.794000
✓ 2018-01-13 05:10:32.794000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 05:10:32.840000
✓ 2018-01-13 05:10:32.847000
✓ 2018-01-13 05:10:32.862000
✓ 2018-01-13 05:10:32.871000
✓ 2018-01-13 05:10:32.886000
✓ 2018-01-13 05:10:32.898000
✓ 2018-01-13 05:10:32.909000
✓ 2018-01-13 05:10:32.920000
✓ 2018-01-13 05:10:32.929000
✓ 2018-01-13 05:10:32.944000
✓ 2018-01-13 05:10:32.951000
✓ 2018-01-13 05:10:32.957000
✓ 2018-01-13 05:10:32

✓ 2018-01-13 05:10:38.657000
✓ 2018-01-13 05:10:38.666000
✓ 2018-01-13 05:10:38.675000
✓ 2018-01-13 05:10:38.702000
sleeping 0.03 seconds
✓ 2018-01-13 05:10:38.744000
sleeping 0.02 seconds
✓ 2018-01-13 05:10:38.773000
sleeping 0.05 seconds
✓ 2018-01-13 05:10:38.832000
✓ 2018-01-13 05:10:38.837000
✓ 2018-01-13 05:10:38.847000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 05:10:38.901000
✓ 2018-01-13 05:10:38.917000
✓ 1970-01-01 00:00:00
sleeping 0.01 seconds
✓ 2018-01-13 05:10:38.973000
sleeping 0.03 seconds
✓ 2018-01-13 05:10:39.021000
✓ 2018-01-13 05:10:39.027000
✓ 2018-01-13 05:10:39.036000
✓ 2018-01-13 05:10:39.046000
✓ 2018-01-13 05:10:39.057000
✓ 2018-01-13 05:10:39.068000
✓ 2018-01-13 05:10:39.078000
✓ 2018-01-13 05:10:39.085000
✓ 1970-01-01 00:00:00
✓ 2018-01-13 05:10:39.097000
✓ 2018-01-13 05:10:39.114000
sleeping 0.09 seconds
✓ 2018-01-13 05:10:39.264000
✓ 2018-01-13 05:10:39.269000
✓ 2018-01-13 05:10:39.278000
✓ 2018-01-13 05:10:39.296000
✓ 2018-01-13 05:10:39.321000
sleeping 0.02 secon

KeyboardInterrupt: 